# Modify metadata in iRODS - split out date and time for easier search

## Set up

In [ ]:
from irods.session import iRODSSession

cred_path = "D:/sbranchett/My Documents/Python Scripts/iRods_credentials.txt" #path to file containing iRODS credentials
irods_home = "/tempZone/home/scottc/ProRail_iRODS" #iRODS path to start from

## Get an iRODS session

In [ ]:
creds = [line.rstrip('\n') for line in open(cred_path, "r")]

with iRODSSession(host=creds[0], port=creds[1], user=creds[2], password=creds[3], zone=creds[4], \
                  client_server_negotiation = "request_server_negotiation", client_server_policy = "CS_NEG_REQUIRE", \
                  encryption_algorithm = "AES-256-CBC", encryption_key_size = 32, encryption_num_hash_rounds = 16, \
                  encryption_salt_size = 8) as session:
    print ('Got A Session', session)
    pass

coll = session.collections.get(irods_home)

## Loop over collections

In [ ]:
for col in coll.subcollections:
    print(col.name)
    for item in col.metadata.items():
        if item.name == "collection_stamp": # split out date and time for collection
            collection_time = item.value[11:]
            collection_date = item.value[:10]
            try:
                col.metadata.add("collection_time",collection_time)
            except:
                print("could not add metadata collection_time "+collection_time)
            try:
                col.metadata.add("collection_date",collection_date)
            except:
                print("could not add metadata collection_date "+collection_date)
            try:
                col.metadata.remove(item.name, item.value, item.units)
            except:
                print("could not delete metadata "+str(col.name)+" "+str(item.name)+" "+str(item.value)+" "+str(item.units))
        elif item.name == "event_time_stamp": # split out date and time for event
            event_time = item.value[11:]
            event_date = item.value[:10]
            try:
                col.metadata.add("event_time",event_time)
            except:
                print("could not add metadata event_time "+event_time)
            try:
                col.metadata.add("event_date",event_date)
            except:
                print("could not add metadata event_date "+event_date)
            try:
                col.metadata.remove(item.name, item.value, item.units)
            except:
                print("could not delete metadata "+str(col.name)+" "+str(item.name)+" "+str(item.value)+" "+str(item.units))
    for obj in col.data_objects: # add a flag for presence of Webcam dat
        if "Webcam" in obj.name and ".avi" in obj.name:
            try:
                col.metadata.add("webcam_movie","yes")
            except:
                print("could not add metadata webcam_movie yes")